In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("/workspaces/TSA-Data_Science-24-25/raw data/ACSST5Y2010.S1901-2025-01-10T102329.csv")

df.iloc[11, [1, 9, 17, 25, 33, 41, 49, 57, 65, 73, 81, 89]]

Autauga County, Alabama!!Households!!Estimate     53,255
Baldwin County, Alabama!!Households!!Estimate     50,147
Barbour County, Alabama!!Households!!Estimate     33,219
Bibb County, Alabama!!Households!!Estimate        41,770
Blount County, Alabama!!Households!!Estimate      45,549
Bullock County, Alabama!!Households!!Estimate     31,602
Butler County, Alabama!!Households!!Estimate      30,659
Calhoun County, Alabama!!Households!!Estimate     38,407
Chambers County, Alabama!!Households!!Estimate    31,467
Cherokee County, Alabama!!Households!!Estimate    40,690
Chilton County, Alabama!!Households!!Estimate     39,486
Choctaw County, Alabama!!Households!!Estimate     31,076
Name: 11, dtype: object

In [6]:
# Dictionary of state names to abbreviations
state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY',
    'District of Columbia': 'DC'
}

# Get the relevant column indices (every 8th column starting from index 1)
relevant_cols = list(range(1, df.shape[1], 8))
print("Relevant Columns:", relevant_cols)

# Initialize lists to store our data
states = []
counties = []
metrics = []

# Extract data for each relevant column
for col_idx in relevant_cols:
    # Get column name and split it to extract location info
    col_name = df.columns[col_idx]
    print(f"Processing Column {col_idx}: {col_name}")

    location_info = col_name.split('!!')[0]  # Split by !! to remove other info
    county, state = location_info.split(',')  # Split county and state
    
    # Clean up by stripping whitespace
    county = county.strip()
    state = state.strip()
    
    # Convert state to abbreviation if it exists in our dictionary
    state_abbr = state_abbrev.get(state, state)  # Use original name if not found
    
    # Get the metric from row 11 (12th row)
    metric = df.iloc[11, col_idx]
    print(f"Extracted Data - County: {county}, State: {state}, Metric: {metric}")
    
    # Append to our lists
    counties.append(county)
    states.append(state_abbr)
    metrics.append(metric)

# Create new DataFrame
income_df = pd.DataFrame({
    'State': states,
    'County': counties,
    'Metric': metrics
})

# Remove Puerto Rico rows
income_df = income_df[income_df['State'] != 'Puerto Rico']

income_df.head()

Relevant Columns: [1, 9, 17, 25, 33, 41, 49, 57, 65, 73, 81, 89, 97, 105, 113, 121, 129, 137, 145, 153, 161, 169, 177, 185, 193, 201, 209, 217, 225, 233, 241, 249, 257, 265, 273, 281, 289, 297, 305, 313, 321, 329, 337, 345, 353, 361, 369, 377, 385, 393, 401, 409, 417, 425, 433, 441, 449, 457, 465, 473, 481, 489, 497, 505, 513, 521, 529, 537, 545, 553, 561, 569, 577, 585, 593, 601, 609, 617, 625, 633, 641, 649, 657, 665, 673, 681, 689, 697, 705, 713, 721, 729, 737, 745, 753, 761, 769, 777, 785, 793, 801, 809, 817, 825, 833, 841, 849, 857, 865, 873, 881, 889, 897, 905, 913, 921, 929, 937, 945, 953, 961, 969, 977, 985, 993, 1001, 1009, 1017, 1025, 1033, 1041, 1049, 1057, 1065, 1073, 1081, 1089, 1097, 1105, 1113, 1121, 1129, 1137, 1145, 1153, 1161, 1169, 1177, 1185, 1193, 1201, 1209, 1217, 1225, 1233, 1241, 1249, 1257, 1265, 1273, 1281, 1289, 1297, 1305, 1313, 1321, 1329, 1337, 1345, 1353, 1361, 1369, 1377, 1385, 1393, 1401, 1409, 1417, 1425, 1433, 1441, 1449, 1457, 1465, 1473, 1481, 1489,

,State,County,Metric
0,AL,Autauga County,"53,255"
1,AL,Baldwin County,"50,147"
2,AL,Barbour County,"33,219"
3,AL,Bibb County,"41,770"
4,AL,Blount County,"45,549"


In [11]:
# Ensure that the 'Metric' column is converted to string, remove commas, and convert back to int
income_df['Metric'] = income_df['Metric'].astype(str).str.replace(',', '').astype(int)

# Assign levels based on percentile bins
income_df['Income Level'] = pd.qcut(
    income_df['Metric'],
    q=[0, 0.2, 0.4, 0.6, 0.8, 1],   # Define the percentiles
    labels=[5, 4, 3, 2, 1]          # Labels for each percentile group
).astype(int)

income_df.head()

,State,County,Metric,Income Level
0,AL,Autauga County,53255,1
1,AL,Baldwin County,50147,2
2,AL,Barbour County,33219,5
3,AL,Bibb County,41770,3
4,AL,Blount County,45549,2


In [12]:
income_df

,State,County,Metric,Income Level
0,AL,Autauga County,53255,1
1,AL,Baldwin County,50147,2
2,AL,Barbour County,33219,5
3,AL,Bibb County,41770,3
4,AL,Blount County,45549,2
...,...,...,...,...
3138,WY,Sweetwater County,69828,1
3139,WY,Teton County,70271,1
3140,WY,Uinta County,58346,1
3141,WY,Washakie County,48379,2


In [21]:
df = pd.read_excel("/workspaces/TSA-Data_Science-24-25/raw data/DECENNIALDHC2020.P9-2025-01-10T101403.xlsx", sheet_name="Data")

df.iloc[:11, :2]

,Label,"Autauga County, Alabama"
0,Total:,"58,805"
1,Hispanic or Latino,"2,117"
2,Not Hispanic or Latino:,"56,688"
3,Population of one race:,"54,198"
4,White alone,"41,582"
5,Black or African American alone,"11,352"
6,American Indian and Alaska Native alone,184
7,Asian alone,873
8,Native Hawaiian and Other Pacific Islander alone,22
9,Some Other Race alone,185


In [65]:
states = []
counties = []
top_races = []

for column in df.columns[1:]:
    # Extract county and state from the column name (split by comma)
    county, state = column.split(", ")
    state_abbr = state_abbrev.get(state, state)
    
    # Extract the population numbers (all rows except 'Label' row)
    population_data = df[column].values

    # Convert each value in the population_data to a string, remove commas, and convert back to int
    population_data = [int(str(val).replace(',', '')) for val in population_data]

    # Get population values for each race category
    total, races = population_data[0], [population_data[4], population_data[5], 
    population_data[1], population_data[7], population_data[8], 
    population_data[6], population_data[9] + population_data[10]]

    # Calculate percentages and labels for each race group
    race_percentages = [round(races[0] / total * 100), round(races[1] / total * 100), round(races[2] / total * 100), 
                        round(races[3] / total * 100), round(races[4] / total * 100), round(races[5] / total * 100), 
                        round(races[6] / total * 100)]
    race_labels = ['W', 'B', 'H', 'A', 'P', 'N', 'O']

    # Create a list of (percentage, race_label) tuples
    race_data = [(race_percentages[i], race_labels[i]) for i in range(len(race_percentages))]

    # Sort the list of race data by percentage in descending order
    sorted_race_data = sorted(race_data, key=lambda x: x[0], reverse=True)
    
    # Build the race string by iterating over the sorted list and adding races above the 30% threshold
    race_string = ""
    for percentage, race_label in sorted_race_data:
        if percentage >= 30:
            race_string += race_label
    if race_string == "":
        race_string = "E"
    
    # Append to the list
    top_races.append(race_string)
    states.append(state_abbr)
    counties.append(county)

# Create the new dataframe with the state, county, and the top races
race_df = pd.DataFrame({
    'State': states,
    'County': counties,
    'Top Races': top_races
})

# Remove Puerto Rico rows
race_df = race_df[race_df['State'] != 'Puerto Rico']

race_df.head()

,State,County,Top Races
0,AL,Autauga County,W
1,AL,Baldwin County,W
2,AL,Barbour County,BW
3,AL,Bibb County,W
4,AL,Blount County,W


In [66]:
race_df

,State,County,Top Races
0,AL,Autauga County,W
1,AL,Baldwin County,W
2,AL,Barbour County,BW
3,AL,Bibb County,W
4,AL,Blount County,W
...,...,...,...
3138,WY,Sweetwater County,W
3139,WY,Teton County,W
3140,WY,Uinta County,W
3141,WY,Washakie County,W


In [67]:
lowtier_df = pd.read_csv("/workspaces/TSA-Data_Science-24-25/raw data/City_zhvi_uc_sfrcondo_tier_0.0_0.33_sm_sa_month.csv")
midtier_df = pd.read_csv("/workspaces/TSA-Data_Science-24-25/raw data/City_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month (1).csv")
hightier_df = pd.read_csv("/workspaces/TSA-Data_Science-24-25/raw data/City_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month.csv")

print("lowtier_df:\n", lowtier_df.head())
print("midtier_df:\n", midtier_df.head())
print("hightier_df:\n", hightier_df.head()) 

lowtier_df:
   State                      Metropolitan Area              County  \
0    NY  New York-Newark-Jersey City, NY-NJ-PA       Queens County   
1    CA     Los Angeles-Long Beach-Anaheim, CA  Los Angeles County   
2    TX   Houston-The Woodlands-Sugar Land, TX       Harris County   
3    IL     Chicago-Naperville-Elgin, IL-IN-WI         Cook County   
4    TX          San Antonio-New Braunfels, TX        Bexar County   

          City        Jan-06        Feb-06        Mar-06        Apr-06  \
0     New York  271773.15940  275366.63600  277944.33220  280244.01150   
1  Los Angeles  324969.88010  328290.38720  331652.70670  334440.63130   
2      Houston   70025.82616   69974.22515   69940.54027   69983.11000   
3      Chicago  126133.15390  127280.17840  128340.06990  129284.15440   
4  San Antonio   66039.45972   66238.56018   66431.74135   66676.58333   

        May-06        Jun-06  ...        Mar-10        Apr-10        May-10  \
0  282808.5711  285603.96950  ...  256626.

In [68]:
# Merge income_df and race_df on State and County
combined_income_race = pd.merge(income_df, race_df, on=["State", "County"], how="inner")
combined_income_race = combined_income_race.rename(columns={"Top Races": "Racial Groups"})
combined_income_race.drop('Metric', axis=1, inplace=True)

combined_income_race.head()

,State,County,Income Level,Racial Groups
0,AL,Autauga County,1,W
1,AL,Baldwin County,2,W
2,AL,Barbour County,5,BW
3,AL,Bibb County,3,W
4,AL,Blount County,2,W


In [69]:
# Renaming columns for lowtier, midtier, and hightier datasets
def rename(df, type):
    rename_key = {col: f"{col} ZHVI | {type}" for col in df.columns if col not in ['State', 'Metropolitan Area', 'County', 'City']}
    return df.rename(columns=rename_key)

lowtier_df = rename(lowtier_df, "Low-Tier")
midtier_df = rename(midtier_df, "Mid-Tier")
hightier_df = rename(hightier_df, "High-Tier")

In [70]:
# Merge lowtier_df, midtier_df, and hightier_df on State, County, and City
low_mid_df = pd.merge(lowtier_df, midtier_df, on=["State", "Metropolitan Area", "County", "City"], how="inner")
low_mid_high_df = pd.merge(low_mid_df, hightier_df, on=["State", "Metropolitan Area", "County", "City"], how="inner")
low_mid_high_df = low_mid_high_df.dropna()
print(low_mid_high_df.columns.to_list())
low_mid_high_df

['State', 'Metropolitan Area', 'County', 'City', 'Jan-06 ZHVI | Low-Tier', 'Feb-06 ZHVI | Low-Tier', 'Mar-06 ZHVI | Low-Tier', 'Apr-06 ZHVI | Low-Tier', 'May-06 ZHVI | Low-Tier', 'Jun-06 ZHVI | Low-Tier', 'Jul-06 ZHVI | Low-Tier', 'Aug-06 ZHVI | Low-Tier', 'Sep-06 ZHVI | Low-Tier', 'Oct-06 ZHVI | Low-Tier', 'Nov-06 ZHVI | Low-Tier', 'Dec-06 ZHVI | Low-Tier', 'Jan-07 ZHVI | Low-Tier', 'Feb-07 ZHVI | Low-Tier', 'Mar-07 ZHVI | Low-Tier', 'Apr-07 ZHVI | Low-Tier', 'May-07 ZHVI | Low-Tier', 'Jun-07 ZHVI | Low-Tier', 'Jul-07 ZHVI | Low-Tier', 'Aug-07 ZHVI | Low-Tier', 'Sep-07 ZHVI | Low-Tier', 'Oct-07 ZHVI | Low-Tier', 'Nov-07 ZHVI | Low-Tier', 'Dec-07 ZHVI | Low-Tier', 'Jan-08 ZHVI | Low-Tier', 'Feb-08 ZHVI | Low-Tier', 'Mar-08 ZHVI | Low-Tier', 'Apr-08 ZHVI | Low-Tier', 'May-08 ZHVI | Low-Tier', 'Jun-08 ZHVI | Low-Tier', 'Jul-08 ZHVI | Low-Tier', 'Aug-08 ZHVI | Low-Tier', 'Sep-08 ZHVI | Low-Tier', 'Oct-08 ZHVI | Low-Tier', 'Nov-08 ZHVI | Low-Tier', 'Dec-08 ZHVI | Low-Tier', 'Jan-09 ZHVI | 

,State,Metropolitan Area,County,City,Jan-06 ZHVI | Low-Tier,Feb-06 ZHVI | Low-Tier,Mar-06 ZHVI | Low-Tier,Apr-06 ZHVI | Low-Tier,May-06 ZHVI | Low-Tier,Jun-06 ZHVI | Low-Tier,...,Mar-10 ZHVI | High-Tier,Apr-10 ZHVI | High-Tier,May-10 ZHVI | High-Tier,Jun-10 ZHVI | High-Tier,Jul-10 ZHVI | High-Tier,Aug-10 ZHVI | High-Tier,Sep-10 ZHVI | High-Tier,Oct-10 ZHVI | High-Tier,Nov-10 ZHVI | High-Tier,Dec-10 ZHVI | High-Tier
0,NY,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,New York,271773.15940,275366.63600,277944.33220,280244.01150,282808.57110,285603.96950,...,1.100110e+06,1.098160e+06,1.096268e+06,1.099103e+06,1.104840e+06,1.109851e+06,1.114661e+06,1.117419e+06,1.117309e+06,1.114193e+06
1,CA,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,Los Angeles,324969.88010,328290.38720,331652.70670,334440.63130,337917.33190,341013.50950,...,7.767155e+05,7.795356e+05,7.856631e+05,7.853399e+05,7.823003e+05,7.746588e+05,7.656140e+05,7.566198e+05,7.483628e+05,7.452383e+05
2,TX,"Houston-The Woodlands-Sugar Land, TX",Harris County,Houston,70025.82616,69974.22515,69940.54027,69983.11000,70288.00590,70595.44407,...,2.626782e+05,2.636077e+05,2.644418e+05,2.644407e+05,2.633443e+05,2.610592e+05,2.589025e+05,2.571945e+05,2.563344e+05,2.556453e+05
3,IL,"Chicago-Naperville-Elgin, IL-IN-WI",Cook County,Chicago,126133.15390,127280.17840,128340.06990,129284.15440,130075.74690,130544.30160,...,4.461005e+05,4.470648e+05,4.458909e+05,4.475403e+05,4.431160e+05,4.395150e+05,4.336614e+05,4.290512e+05,4.232324e+05,4.176005e+05
4,TX,"San Antonio-New Braunfels, TX",Bexar County,San Antonio,66039.45972,66238.56018,66431.74135,66676.58333,66998.49430,67434.80978,...,2.117142e+05,2.121381e+05,2.125702e+05,2.126809e+05,2.126695e+05,2.127269e+05,2.126661e+05,2.121040e+05,2.108790e+05,2.096505e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19766,MN,"Grand Rapids, MN",Itasca County,Bovey,48123.86595,48307.28283,48530.84714,48738.71184,48649.18504,48474.22314,...,1.192620e+05,1.193986e+05,1.192972e+05,1.193781e+05,1.190902e+05,1.188066e+05,1.183497e+05,1.170413e+05,1.151491e+05,1.127336e+05
19768,WA,"Spokane-Spokane Valley, WA",Spokane County,Fairfield,55232.04160,55875.95362,56575.31816,57563.78531,59129.06629,60655.45036,...,1.836043e+05,1.854356e+05,1.865931e+05,1.864931e+05,1.836202e+05,1.806280e+05,1.767468e+05,1.738648e+05,1.694534e+05,1.656952e+05
19780,WV,"Wheeling, WV-OH",Ohio County,Clearview,81604.33624,81673.79567,81383.34490,81363.58644,81993.50350,82645.17751,...,1.619863e+05,1.615578e+05,1.610764e+05,1.611784e+05,1.617382e+05,1.625353e+05,1.626768e+05,1.623991e+05,1.618883e+05,1.615857e+05
19781,NV,"Las Vegas-Henderson-Paradise, NV",Clark County,Mount Charleston,228238.15170,229949.93900,231512.17300,232563.45860,231784.77040,231454.05070,...,5.042237e+05,4.911814e+05,4.779022e+05,4.668564e+05,4.653403e+05,4.649443e+05,4.583435e+05,4.494842e+05,4.415563e+05,4.368349e+05


In [71]:
# Merge everything together
combined_df = pd.merge(low_mid_high_df, combined_income_race, on=["State", "County"], how='left')

print(combined_df.columns.to_list())
combined_df

['State', 'Metropolitan Area', 'County', 'City', 'Jan-06 ZHVI | Low-Tier', 'Feb-06 ZHVI | Low-Tier', 'Mar-06 ZHVI | Low-Tier', 'Apr-06 ZHVI | Low-Tier', 'May-06 ZHVI | Low-Tier', 'Jun-06 ZHVI | Low-Tier', 'Jul-06 ZHVI | Low-Tier', 'Aug-06 ZHVI | Low-Tier', 'Sep-06 ZHVI | Low-Tier', 'Oct-06 ZHVI | Low-Tier', 'Nov-06 ZHVI | Low-Tier', 'Dec-06 ZHVI | Low-Tier', 'Jan-07 ZHVI | Low-Tier', 'Feb-07 ZHVI | Low-Tier', 'Mar-07 ZHVI | Low-Tier', 'Apr-07 ZHVI | Low-Tier', 'May-07 ZHVI | Low-Tier', 'Jun-07 ZHVI | Low-Tier', 'Jul-07 ZHVI | Low-Tier', 'Aug-07 ZHVI | Low-Tier', 'Sep-07 ZHVI | Low-Tier', 'Oct-07 ZHVI | Low-Tier', 'Nov-07 ZHVI | Low-Tier', 'Dec-07 ZHVI | Low-Tier', 'Jan-08 ZHVI | Low-Tier', 'Feb-08 ZHVI | Low-Tier', 'Mar-08 ZHVI | Low-Tier', 'Apr-08 ZHVI | Low-Tier', 'May-08 ZHVI | Low-Tier', 'Jun-08 ZHVI | Low-Tier', 'Jul-08 ZHVI | Low-Tier', 'Aug-08 ZHVI | Low-Tier', 'Sep-08 ZHVI | Low-Tier', 'Oct-08 ZHVI | Low-Tier', 'Nov-08 ZHVI | Low-Tier', 'Dec-08 ZHVI | Low-Tier', 'Jan-09 ZHVI | 

,State,Metropolitan Area,County,City,Jan-06 ZHVI | Low-Tier,Feb-06 ZHVI | Low-Tier,Mar-06 ZHVI | Low-Tier,Apr-06 ZHVI | Low-Tier,May-06 ZHVI | Low-Tier,Jun-06 ZHVI | Low-Tier,...,May-10 ZHVI | High-Tier,Jun-10 ZHVI | High-Tier,Jul-10 ZHVI | High-Tier,Aug-10 ZHVI | High-Tier,Sep-10 ZHVI | High-Tier,Oct-10 ZHVI | High-Tier,Nov-10 ZHVI | High-Tier,Dec-10 ZHVI | High-Tier,Income Level,Racial Groups
0,NY,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,New York,271773.15940,275366.63600,277944.33220,280244.01150,282808.57110,285603.96950,...,1.096268e+06,1.099103e+06,1.104840e+06,1.109851e+06,1.114661e+06,1.117419e+06,1.117309e+06,1.114193e+06,1.0,E
1,CA,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,Los Angeles,324969.88010,328290.38720,331652.70670,334440.63130,337917.33190,341013.50950,...,7.856631e+05,7.853399e+05,7.823003e+05,7.746588e+05,7.656140e+05,7.566198e+05,7.483628e+05,7.452383e+05,1.0,H
2,TX,"Houston-The Woodlands-Sugar Land, TX",Harris County,Houston,70025.82616,69974.22515,69940.54027,69983.11000,70288.00590,70595.44407,...,2.644418e+05,2.644407e+05,2.633443e+05,2.610592e+05,2.589025e+05,2.571945e+05,2.563344e+05,2.556453e+05,1.0,H
3,IL,"Chicago-Naperville-Elgin, IL-IN-WI",Cook County,Chicago,126133.15390,127280.17840,128340.06990,129284.15440,130075.74690,130544.30160,...,4.458909e+05,4.475403e+05,4.431160e+05,4.395150e+05,4.336614e+05,4.290512e+05,4.232324e+05,4.176005e+05,1.0,W
4,TX,"San Antonio-New Braunfels, TX",Bexar County,San Antonio,66039.45972,66238.56018,66431.74135,66676.58333,66998.49430,67434.80978,...,2.125702e+05,2.126809e+05,2.126695e+05,2.127269e+05,2.126661e+05,2.121040e+05,2.108790e+05,2.096505e+05,2.0,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8669,MN,"Grand Rapids, MN",Itasca County,Bovey,48123.86595,48307.28283,48530.84714,48738.71184,48649.18504,48474.22314,...,1.192972e+05,1.193781e+05,1.190902e+05,1.188066e+05,1.183497e+05,1.170413e+05,1.151491e+05,1.127336e+05,2.0,W
8670,WA,"Spokane-Spokane Valley, WA",Spokane County,Fairfield,55232.04160,55875.95362,56575.31816,57563.78531,59129.06629,60655.45036,...,1.865931e+05,1.864931e+05,1.836202e+05,1.806280e+05,1.767468e+05,1.738648e+05,1.694534e+05,1.656952e+05,2.0,W
8671,WV,"Wheeling, WV-OH",Ohio County,Clearview,81604.33624,81673.79567,81383.34490,81363.58644,81993.50350,82645.17751,...,1.610764e+05,1.611784e+05,1.617382e+05,1.625353e+05,1.626768e+05,1.623991e+05,1.618883e+05,1.615857e+05,4.0,W
8672,NV,"Las Vegas-Henderson-Paradise, NV",Clark County,Mount Charleston,228238.15170,229949.93900,231512.17300,232563.45860,231784.77040,231454.05070,...,4.779022e+05,4.668564e+05,4.653403e+05,4.649443e+05,4.583435e+05,4.494842e+05,4.415563e+05,4.368349e+05,1.0,WH


In [72]:
combined_df.to_excel("Combined Dataset.xlsx", sheet_name="Data", index=False)